## MVTEc  LOCO  Visualizer

- claculates clip embeddings
- vizualisation via tensorboard projector

from: "https://www.mydrive.ch/shares/48237/1b9106ccdfbb09a0c414bd49fe44a14a/download/430647091-1646842701/mvtec_loco_anomaly_detection.tar.xz"

In [1]:
# activate clip env
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import os
import clip
import torch
import pandas as pd

import warnings
warnings.filterwarnings("ignore", message=".*pandas only supports SQLAlchemy connectable.*")


torch.cuda.empty_cache()
print(f'GPU is available: {torch.cuda.is_available()}')
print("Torch version:", torch.__version__)


from PIL import Image

import torchvision
import torch.utils.tensorboard 
from torch.utils.tensorboard import SummaryWriter


GPU is available: True
Torch version: 2.1.0+cu118


2023-11-12 16:09:01.759098: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-12 16:09:02.041679: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-12 16:09:02.041695: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-12 16:09:02.043123: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-12 16:09:02.177281: I tensorflow/core/platform/cpu_feature_g

In [2]:

def convert_images_for_tensorboard(image_paths):
    # Preprocess the images
    transform_display = torchvision.transforms.Compose([
        torchvision.transforms.Resize(64),  # Resize the images 
        torchvision.transforms.CenterCrop(64),  # Crop the images
        torchvision.transforms.ToTensor()  # Convert images to tensors        
    ])

    images = []  # Array to store the preprocessed images
    for path in image_paths:
        image = Image.open(path).convert('RGB')  # Open the image and convert to RGB
        # Apply the display transformation
        image_display = transform_display(image)
        image_display = image_display.unsqueeze(0) # Add a batch dimension to the image
        images.append(image_display)

    images = torch.cat(images, dim=0)  # Concatenate the images along the batch dimension

    return images

In [4]:
# PATHs
PROJECT_DATA_PATH='/home/bule/projects/MVTec_visualizer/data/mvtec_loco'
LOG_DIR= '/home/bule/projects/MVTec_visualizer/tensorboard_logs'

In [5]:
# paths to images
image_paths = []
for root, dirs, files in os.walk(PROJECT_DATA_PATH):
    if 'ground_truth' not in root:
        for file in files:
            if file.endswith('.png'):
                image_paths.append(os.path.join(root, file))
len(image_paths)
## TODO embedd all images from MVTEC in CLIP space


0

In [6]:
BATCH_SIZE = 1024  # Adjust this based on your GPU memory
FILENAME = 'MVTEC_LOCO_embeddings_df'

if not os.path.exists(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl')):

    # Load the model
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, transform = clip.load('ViT-B/32', device=device)


    # Function to get embeddings for a batch of images
    def get_batch_embeddings(batch_paths):
        images = [Image.open(p) for p in batch_paths]
        tensors = [transform(img) for img in images]
        batch_tensor = torch.stack(tensors).to(device)
        with torch.no_grad():
            batch_features = model.encode_image(batch_tensor)
        return batch_features.cpu().numpy()

    # Calculate embeddings for all images in batches
    all_embeddings = []
    for i in range(0, len(image_paths), BATCH_SIZE):
        batch_paths = image_paths[i:i+BATCH_SIZE]
        batch_embeddings = get_batch_embeddings(batch_paths)
        all_embeddings.extend(batch_embeddings)
        print(f"Processed images {i} to {i + len(batch_embeddings)}")

    # Convert all embeddings to DataFrame and save
    df = pd.DataFrame(all_embeddings, index=image_paths)
    df.to_pickle(os.path.join(PROJECT_DATA_PATH, FILENAME + '.pkl'))
else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

df.head()


OSError: Cannot save file into a non-existent directory: '/home/bule/projects/MVTec_visualizer/data/mvtec_loco'

In [7]:
# make tensorboard logs

print(FILENAME)
TENSORBOARD_LOGS_PATH=os.path.join(LOG_DIR,'_'+FILENAME).replace(" ", "_")


if not  os.path.exists(TENSORBOARD_LOGS_PATH):

    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

    # Convert the resulting series of lists to a NumPy array
    numpy_array = df.to_numpy()

    # resize images 
    images = convert_images_for_tensorboard(image_paths)

    # for tensorboard
    writer = SummaryWriter(TENSORBOARD_LOGS_PATH)
    writer.add_embedding(numpy_array, label_img=images)

else:
    df = pd.read_pickle(os.path.join(PROJECT_DATA_PATH,FILENAME+'.pkl'))

print(f'{FILENAME}  number of images:{len(image_paths)}')

MVTEC_LOCO_embeddings_df
MVTEC_LOCO_embeddings_df  number of images:3651


### run tensorboard on port forward to browser

In [8]:
!tensorboard  --logdir $TENSORBOARD_LOGS_PATH

2023-10-20 13:07:39.932508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-20 13:07:39.932545: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-20 13:07:39.932588: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 13:07:39.944385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 13:07:41.337600: W tensorflow/compiler/